In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
import sys
import os

import sys
import os
sys.path.append("/content/drive/MyDrive/nlp")

Mounted at /content/drive


In [ ]:
sys.path.append("/content/drive/MyDrive/lp")
import vocab_helpers

In [ ]:
# !pip install transformers
import numpy as np
import glob
import xml.etree.cElementTree as ET
import re
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import activations
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import backend as K
import numpy as np
# !pip install transformers

In [ ]:
!pip install emoji
import emoji

In [ ]:
GT    = '/content/drive/MyDrive/nlp/truth.txt'
true_values = {}
f=open(GT)
for line in f:
    linev = line.strip().split(":::")
    true_values[linev[0]] = linev[1]
f.close()

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords = stopwords.words('english')

In [ ]:
def replace_contracted_form(contracted_word, pos, dictionary):
    long_form = []
    if "'" in contracted_word:
        # print("Found apostrophe in word: ", contracted_word, ' with pos: ', pos)
        split_words = contracted_word.split("'")
        check_if_in_dict = False
        # If the contraction is a nominal + verbal or a proper noun + verbal
        if pos is 'L' or pos is 'M':
            long_form.append(split_words[0])
            if split_words[1].lower() in contractions:
                long_form.extend(contractions[split_words[1].lower()].split())
        # If the contraction is a whole verb (like let's or isn't)
        elif pos in ['V', 'Y', 'O'] and contracted_word.lower() in contractions:
            long_form.extend(contractions[contracted_word.lower()].split())
        # If the contraction is proper noun with possessive or a nominal with a possessive or even a (proper) noun
        elif pos in ['S', 'Z', 'D', 'N', '^']:
            if contracted_word.lower() in contractions:
                long_form.extend(contractions[contracted_word.lower()].split())
            elif split_words[1].lower() == 's':
                long_form.append(split_words[0])
            elif contracted_word.lower() in contractions:
                long_form.extend(contractions[contracted_word.lower()].split())
            else:
                check_if_in_dict = True
        # Can skip ' which are just punctuation marks (usually used to emphasize or quote something)
        elif pos is ',':
            # print("Punctuation, nothing to replace.", split_words[0], ' -- ', split_words[1])
            return []
        # Never replace contractions in emojis or emoticons (will be translated later)
        elif pos is 'E':
            long_form.append(contracted_word)
        else:
            check_if_in_dict = True
        if check_if_in_dict:
            # Attempt to separate words which have been separated by ' by human error
            clean0 = re.findall("[a-zA-Z]+", split_words[0])
            clean1 = re.findall("[a-zA-Z]+", split_words[1])
            if clean0 != [] and clean0[0].lower() in dictionary and clean1 != [] and clean1[0].lower() in dictionary:
                # print("Cleaned to ", clean0, ', ', clean1)
                long_form.extend([clean0[0], clean1[0]])
            else:
                # print("Word couldn't be de-contracted!")
                long_form.append(contracted_word)
        return long_form
    else:
        return long_form.append(contracted_word)

In [ ]:
DIREC = "/content/drive/MyDrive/nlp/"

# data cleaning
text1=[]
sent = []
sent_y =[]
y=[]
for file in glob.glob(DIREC+"*.xml"):
    tree = ET.parse(file)
    doc = tree.iter("document")
    l= []
    for d in doc:
        c = d.text
        # c = re.sub(u"\\#.*?\\#", "", c)
        l_str = c.replace("USER", "").replace("URL", "")
        l_str = l_str.strip()
        # l_str = emoji.demojize(l_str)
        sent.append(l_str)
        l.append(l_str)
    a = " ".join(l)
    number = len(l)
    USERCODE = file.split("/")[-1][:-4]
    if true_values[USERCODE] =='NI':
        y.append(0)
        sent_y+=[0]*number
    else:
        y.append(1)
        sent_y+=[1]*number

    text1.append(a)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(list(text1),list(y),test_size = 0.2,shuffle = True,random_state=42)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
import string
from nltk.stem import WordNetLemmatizer
!pip install contractions
from nltk.stem import PorterStemmer
import contractions
from nltk import word_tokenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords + list(string.punctuation)
def remove_stopwords(text):
    output= [i for i in text if i not in stop]
    return output
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
from nltk.stem import PorterStemmer

# init stemmer
porter_stemmer=PorterStemmer()

In [ ]:
def furtherclean(data):
    data_clean = []
    for text in data:
      contract = contractions.fix(text)
      token = word_tokenize(str(contract).lower())
      nostop = remove_stopwords(token)
      stemmed_words=[porter_stemmer.stem(word=word) for word in nostop]
      data_clean.append(stemmed_words)
    return data_clean  

In [ ]:
clean_train = furtherclean(x_train)
clean_test = furtherclean(x_test)


In [ ]:
# len(clean)

In [ ]:
#
def posfeature (data):

    pos_tag = []
    for text in data:
        pos = nltk.pos_tag(text)
        output = [tuple[1] for tuple in pos]
        pos_tag.append(' '.join(output))

    tf_pos = TfidfVectorizer(ngram_range=(1, 1), max_features=30, norm='l2') 
    tf_pos_feature = tf_pos.fit_transform(pos_tag)
    tf_pos_feature = tf_pos_feature.toarray()
    return tf_pos_feature

In [ ]:
posfeature_train = posfeature(clean_train)
posfeature_test = posfeature(clean_test)

In [ ]:
#semantic
def tf_idffeature (data):
    full = []
    for text in data:
        text = ' '.join(str(x) for x in text)
        full.append(text)
    # tfidf feature:
        # character based:
    tf_char = TfidfVectorizer(analyzer='char_wb', ngram_range=(1, 3), max_features=32, norm='l2')
    tf_char_feature = tf_char.fit_transform(full).toarray()
        # word based
    tf_word = TfidfVectorizer(ngram_range=(1, 3), max_features=50, norm='l2')
    tf_word_feature = tf_word.fit_transform(full).toarray()

    return tf_char_feature,tf_word_feature

In [ ]:
tf_char_train,tf_word_train = tf_idffeature(clean_train)
tf_char_test,tf_word_test = tf_idffeature(clean_test)


In [ ]:
# import os
# path="/content/drive/MyDrive/nlp/vocab_helpers"
# os.chdir(path)
# os.listdir(path)

import emoji, re, time, os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import sentiwordnet as swn
from nltk import ngrams
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import LdaModel
from gensim.corpora import MmCorpus, Dictionary
import vocab_helpers as helper

In [ ]:
def build_subj_dicionary(lines):
    subj_dict = dict()
    for line in lines:
        splits = line.split(' ')
        if len(splits) == 6:
            word = splits[2][6:]        # the word analyzed
            word_type = splits[0][5:]   # weak or strong subjective
            pos = splits[3][5:]         # part of speech: noun, verb, adj, adv or anypos
            polarity = splits[5][14:]   # its polarity: can be positive, negative or neutral
            new_dict_entry = {pos: [word_type, polarity]}
            if word in subj_dict.keys():
                subj_dict[word].update(new_dict_entry)
            else:
                subj_dict[word] = new_dict_entry
    return subj_dict

In [ ]:
nltk.download('punkt')
def normal_feature(data):
  text_normal_feature=[]
  for tweet_tokens in data:
    tweet_tokens=word_tokenize(tweet_tokens)
    capitalized_words = user_specific = intensifiers = tweet_len_ch = 0
    for t in tweet_tokens:
            tweet_len_ch += len(t)
            if t.isupper() and len(t) > 1:
                capitalized_words += 1       # count of capitalized words
            if t.startswith("@"):
                user_specific += 1          # count of user mentions
            if t.startswith("#"):
                user_specific += 1          # count-based feature of hashtags used (excluding sarcasm or sarcastic)
            if t.lower().startswith("haha") or re.match('l(o)+l$', t.lower()):
                user_specific += 1          # binary feature marking the presence of laughter
            if t in helper.strong_negations:
                intensifiers += 1           # count-based feature of strong negations
            if t in helper.strong_affirmatives:
                intensifiers += 1           # count-based feature of strong affirmatives
            if t in helper.interjections:
                intensifiers += 1           # count-based feature of relevant interjections
            if t in helper.intensifiers:
                intensifiers += 1           # count-based feature of relevant intensifiers
            if t in helper.punctuation:
                user_specific += 1          # count-based feature of relevant punctuation signs
            if t in emoji.UNICODE_EMOJI:
                user_specific += 1          # count-based feature of emojis
    tweet_len_tokens = len(tweet_tokens)  # get the length of the tweet in tokens
    average_token_length = float(tweet_len_tokens) / max(1.0, float(tweet_len_ch))  # average tweet length
    feature_list = {'tw_len_ch': tweet_len_ch, 'tw_len_tok': tweet_len_tokens, 'avg_len': average_token_length,
                    'capitalized': capitalized_words, 'user_specific': user_specific, 'intensifiers': intensifiers}
    text_normal_feature.append(feature_list)
  return text_normal_feature


In [ ]:
f = open('/content/drive/MyDrive/nlp/emoji/emoji_sentiment_dictionary.txt')
lines = f.readlines()      #读取全部内容 ，并以列表方式返回
emojii={}
for line in lines:
  b=[]
  l=line.split()
  # a=word_tokenize(l[4:])
  b.append(l[0])
  b.append(float(l[1]))
  b.append(float(l[2]))
  b.append(float(l[2]))
  b.append(" ".join(l[4:]))
  emojii[b[0]]=b[1:]

In [ ]:
file = open('/content/drive/MyDrive/nlp/subjectivity_lexicon.tff','r')
text = file.read()
lexicon = text.split("\n")
subj_dict = build_subj_dicionary(lexicon)


In [ ]:
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('vader_lexicon')
def senti_feature(data,subj_dict,emojii):
  text_senti_feature=[]

  for tweet_token in data:
    tweet_tokens=word_tokenize(tweet_token)
  #"positive emoji", "negative emoji", "neutral emoji",
    sent_features = dict.fromkeys(["positive emoji", "negative emoji", "neutral emoji",
                                    "subjlexicon weaksubj", "subjlexicon strongsubj",
                                    "subjlexicon positive", "subjlexicon negative",
                                    "subjlexicon neutral", "total sentiment words",
                                    "swn pos", "swn neg", "swn obj"], 0.0)
    for t in tweet_tokens:
    
        if t in emojii.keys():
            sent_features['negative emoji'] += float(emojii[t][0])
            sent_features['neutral emoji'] += float(emojii[t][1])
            sent_features['positive emoji'] += float(emojii[t][2])

    lemmatizer = WordNetLemmatizer()
    pos_translation = {'N': 'noun', 'V': 'verb', 'D': 'adj', 'R': 'adverb'}
    for index in range(len(tweet_tokens)):
        lemmatized = lemmatizer.lemmatize(tweet_tokens[index], 'v')
        if lemmatized in subj_dict.keys():
            if tweet_tokens[index] in pos_translation and pos_translation[tweet_tokens[index]] in subj_dict[lemmatized].keys():
                # Get the type of subjectivity (strong or weak) of this lemmatized word
                sent_features['subjlexicon ' + subj_dict[lemmatized][pos_translation[tweet_tokens[index]]][0]] += 1
                # Get the type of polarity (pos, neg, neutral) of this lemmatized word
                if subj_dict[lemmatized][pos_translation[tweet_tokens[index]]][1] == 'both':
                    sent_features['subjlexicon positive'] += 1
                    sent_features['subjlexicon negative'] += 1
                else:
                    sent_features['subjlexicon ' + subj_dict[lemmatized][pos_translation[tweet_tokens[index]]][1]] += 1
            else:
                if 'anypos' in subj_dict[lemmatized].keys():
                    # Get the type of subjectivity (strong or weak) of this lemmatized word
                    sent_features['subjlexicon ' + subj_dict[lemmatized]['anypos'][0]] += 1 # strong or weak subjectivity
                    # Get the type of polarity (pos, neg, neutral) of this lemmatized word
                    if subj_dict[lemmatized]['anypos'][1] == 'both':
                        sent_features['subjlexicon positive'] += 1
                        sent_features['subjlexicon negative'] += 1
                    else:
                        sent_features['subjlexicon ' + subj_dict[lemmatized]['anypos'][1]] += 1

    # Use the total number of sentiment words as a feature
    sent_features["total sentiment words"] = sent_features["subjlexicon positive"] \
                                              + sent_features["subjlexicon negative"] \
                                              + sent_features["subjlexicon neutral"]
    # Obtain average of all sentiment words (pos, ne, obj) using SentiWordNet Interface
    pos_translation = {'N': 'n', 'V': 'v', 'D': 'a', 'R': 'r'}
    for index in range(len(tweet_tokens)):
        lemmatized = lemmatizer.lemmatize(tweet_tokens[index], 'v')
        if tweet_tokens[index] in pos_translation:
            synsets = list(swn.senti_synsets(lemmatized, pos_translation[tweet_tokens[index]]))
            pos_score = 0
            neg_score = 0
            obj_score = 0
            if len(synsets) > 0:
                for syn in synsets:
                    pos_score += syn.pos_score()
                    neg_score += syn.neg_score()
                    obj_score += syn.obj_score()
                sent_features["swn pos"] = pos_score / float(len(synsets))
                sent_features["swn neg"] = neg_score / float(len(synsets))
                sent_features["swn obj"] = obj_score / float(len(synsets))

    # Vader Sentiment Analyser
    # Obtain the negative, positive, neutral and compound scores of a tweet
    sia = SentimentIntensityAnalyzer()
    polarity_scores = sia.polarity_scores(tweet_token)
    for name, score in polarity_scores.items():
        sent_features["Vader score " + name] = score
    text_senti_feature.append(sent_features)
  return text_senti_feature

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

In [ ]:
f = open('/content/drive/MyDrive/nlp/glove.6B.50d.txt')
wordmap = {}
for line in f:
    values = line.split()
    word = values[0]
    weights = np.asarray(values[1:], dtype='float32')
    wordmap[word] = weights
f.close()

In [ ]:
def load_emb(data, wordmap, embedding_size=50, embedding_vocab=400000, init_emb=None):
    # if init_emb is None:
    #     emb_mat = np.random.randn(len(w2i), embedding_size)
    # else:
    #     emb_mat = init_emb
    # f = open(path_to_embedding, 'r')
    found = 0
    em_list=[]
    em_avg=[]
    for text in data:
      tweet_tokens=word_tokenize(text)
      emb_mat=np.zeros([len(word_tokenize(text)), embedding_size])
      for i in range(len(tweet_tokens)):
        if tweet_tokens[i] in wordmap.keys():
            emb_mat[i] = wordmap[tweet_tokens[i]]
            # emb_mat[idx] = np.nan_to_num(vector)
            found += 1
      # for line in f:
      #   content = line.strip().split()
      #   vector = np.asarray(list(map(lambda x: float(np.nan_to_num(x)), content[-embedding_size:])))
      #   print(vector.shape)
      #   word = ' '.join(content[:-embedding_size])

      em_list.append(emb_mat)
      em_avg.append(np.mean(emb_mat,axis=0))
    print(f"Found {found} words in the embedding file.")
    return np.array(em_avg),em_list

In [ ]:
# path_to_embedding='/content/drive/MyDrive/nlp/glove.6B.100d.txt'
# em_avg,em_list=load_emb(text1, wordmap)

In [ ]:
x_train_glove,x_train_glove_list=load_emb(x_train, wordmap)
x_test_glove,x_test_glove_list=load_emb(x_test, wordmap)

In [ ]:
l=[]
for text in text1:
  tweet_tokens=word_tokenize(text)
  l.append(len(tweet_tokens))
  # emb_mat=np.zeros([len(tweet_tokens), 50])
  # for i in range(len(tweet_tokens)):
  #   if tweet_tokens[i] in wordmap.keys():
  #       emb_mat[i] = wordmap[tweet_tokens[i]]
  #       # emb_mat[idx] = np.nan_to_num(vector)


In [ ]:
x_train_glove_1,x_train_glove_list_1=load_emb(x_train, wordmap)

In [ ]:
x_train_glove_1[-1].shape

In [ ]:
x_train_glove_1[1]

In [ ]:
np.savetxt('/content/drive/MyDrive/nlp/x_train_glove.txt',x_train_glove)
np.savetxt('/content/drive/MyDrive/nlp/x_test_glove.txt',x_test_glove)


In [ ]:
x_train_glove.shape

In [ ]:
def cosine_similarity(u, v):
    dot = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u ** 2))
    norm_v = np.sqrt(np.sum(v ** 2))
    cosine_distance = dot / (norm_u * norm_v)
    return cosine_distance
def euclidean_distance(u_vector, v_vector):
    distance = np.sqrt(np.sum([(u - v) ** 2 for u, v in zip(u_vector, v_vector)]))
    return distance
    

In [ ]:
def get_similarity_measures(tweet, vec_map, weighted=False, verbose=False):
    # Filter a bit the tweet so that no punctuation and no stopwords are included
    # filtered_tweet = list(set([w.lower() for w in tweet.split()
    #                            if w.isalnum() and w not in stopwords and w.lower() in vec_map.keys()]))
    # Compute similarity scores between any 2 words in filtered tweet
    similarity_scores_avg = []
    max_words = []
    min_words = []
    for text in tweet:
      similarity_dic = dict.fromkeys(["most similar", "most dissimilar"], 0.0)
      tweet_tokens=word_tokenize(text)
      similarity_scores = []
      max_score = -100
      min_score = 100
      for i in range(len(tweet_tokens)-1):
          wi = tweet_tokens[i]
          for j in range(i + 1, len(tweet_tokens)):
            try:
                wj = text[j]
                similarity = cosine_similarity(vec_map[wi], vec_map[wj])
                if weighted:
                    similarity /= euclidean_distance(vec_map[wi], vec_map[wj])
                similarity_scores.append(similarity)
                if max_score < similarity:
                    max_score = similarity
                    max_words = [wi, wj]
                if min_score > similarity:
                    min_score = similarity
                    min_words = [wi, wj]
            except: continue
      max_score=max(similarity_scores)
      min_score=min(similarity_scores)
      similarity_dic['most similar']= max_score
      similarity_dic['most dissimilar']=min_score
      similarity_scores_avg.append(similarity_dic)
      # if verbose:
      #     print("Filtered tweet: ", text)
      #     if max_score != -100:
      #         print("Maximum similarity is ", max_score, " between words ", max_words)
      #     else:
      #         print("No max! Scores are: ", similarity_scores)
      #     if min_score != 100:
      #         print("Minimum similarity is ", min_score, " between words ", min_words)
      #     else:
      #         print("No min! Scores are: ", similarity_scores)
    return similarity_scores_avg

In [ ]:
def cut(obj, sec):
    return [obj[i:i+sec] for i in range(0,len(obj),sec)]

In [ ]:
clean_sent=furtherclean(sent)

In [ ]:
tweet = clean_sent
vec_map = wordmap
r = cut([i for i in range(84000)], 200)
# weighted = True
similarity_scores_avg = []
for d in r:
    for index in d:
        avg_max_score = 0
        avg_min_score = 0
        sent_no = 200
        max_s = []
        min_s = []
        tweet_tokens = tweet[index]
        similarity_dic = dict.fromkeys(["most similar", "most dissimilar"], 0.0)
        scores = []
        max_score = -100
        min_score = 100
        for i in range(len(tweet_tokens)-1):
            wi = tweet_tokens[i]        
            for j in range(i + 1, len(tweet_tokens)):
                wj = tweet_tokens[j]
                # print(wi,wj)
                if wi in vec_map.keys() and wj in vec_map.keys():

                    similarity = cosine_similarity(vec_map[wi], vec_map[wj])
                    # print(similarity)
                    if max_score < similarity:
                        max_score = similarity
                        max_words = [wi, wj]
                    if min_score > similarity:
                        min_score = similarity
                        min_words = [wi, wj]
                else: continue

  
        # max_score_a=max(scores)
        # min_score_a=min(scores)
        max_s.append(max_score)
        min_s.append(min_score)
    max_avg = (sum(max_s))/sent_no
    min_avg = (sum(min_s))/sent_no
    similarity_dic['most similar']= max_avg
    similarity_dic['most dissimilar']=min_avg
    # similarity_dic['sentence number']=sent_no
    similarity_scores_avg.append(similarity_dic)

In [ ]:
sim_train,sim_test,y_train,y_test = train_test_split(similarity_scores_avg,list(y),test_size = 0.2,shuffle = True,random_state=42)

In [ ]:
from sklearn.feature_extraction import DictVectorizer
def extract_features_from_dict(train_features, test_features):
    # Transform the list of feature-value mappings to a vector
    vector = DictVectorizer(sparse=False)
    # Learn a list of feature name -> indices mappings and transform X_train_features
    x_train_features = vector.fit_transform(train_features).tolist()
    # Just transform the X_test_features, based on the list fitted on X_train_features
    # Disadvantage: named features not encountered during fit_transform will be silently ignored.
    x_test_features = vector.transform(test_features).tolist()
    print('Size of the feature sets: train =  ', len(x_train_features[0]), ', test = ', len(x_test_features[0]))
    return np.nan_to_num(np.array(x_train_features)), np.nan_to_num(np.array(x_test_features))

In [ ]:
from sklearn.feature_extraction import DictVectorizer


In [ ]:
x_train_normal_feature=normal_feature(x_train)
x_test_normal_feature=normal_feature(x_test)

In [ ]:
x_train_senti_feature=senti_feature(x_train,subj_dict,emojii)
x_test_senti_feature=senti_feature(x_test,subj_dict,emojii)

In [ ]:
x_train_sim,x_test_sim=extract_features_from_dict(sim_train,sim_test)

In [ ]:
def merge_dicts(*dict_args):
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

In [ ]:

x_train_normal_feature, x_test_normal_feature=extract_features_from_dict(x_train_normal_feature, x_test_normal_feature)
x_train_senti_feature, x_test_senti_feature=extract_features_from_dict(x_train_senti_feature, x_test_senti_feature)


In [ ]:
# x_train_f = np.hstack((x_train_glove,x_train_feature,tf_word_train,posfeature_train))#tf_word_train,
# x_test_f = np.hstack((x_test_glove,x_test_feature,tf_word_test,posfeature_test))#,tf_char_test
x_train_f = np.hstack((x_train_glove,x_train_normal_feature,x_train_senti_feature,tf_word_train,posfeature_train,x_train_sim))#tf_word_train,
x_test_f = np.hstack((x_test_glove,x_test_normal_feature,x_test_senti_feature,tf_word_test,posfeature_test,x_test_sim))#,tf_char_test

In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertModel
model_name = "bert-large-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
def bert_encoder(review):
    encoded = tokenizer(review, truncation=True, max_length=20000, pad_to_max_length=True)
    return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']

x_train_bert = [bert_encoder(r) for r in x_train]
x_train_bert = np.array(x_train_bert)
x_test_bert = [bert_encoder(r) for r in x_test]
x_test_bert = np.array(x_test_bert)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
#Classifiers
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
#Validation packages
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.model_selection import cross_val_score


In [ ]:
from sklearn import preprocessing
# scaler = preprocessing.StandardScaler().fit(x_train_feature)
# x_train_feature_n=scaler.transform(x_train_feature)
# x_test_feature_n=scaler.transform(x_test_feature)

scaler_normal = preprocessing.StandardScaler().fit(x_train_normal_feature)
x_train_normal_feature_n=scaler_normal.transform(x_train_normal_feature)
x_test_normal_feature_n=scaler_normal.transform(x_test_normal_feature)

scaler_senti = preprocessing.StandardScaler().fit(x_train_senti_feature)
x_train_senti_feature_n=scaler_senti.transform(x_train_senti_feature)
x_test_senti_feature_n=scaler_senti.transform(x_test_senti_feature)

scaler_sim = preprocessing.StandardScaler().fit(x_train_sim)
x_train_sim_n=scaler_sim.transform(x_train_sim)
x_test_sim_n=scaler_sim.transform(x_test_sim)

x_train_f = np.hstack((x_train_glove,x_train_normal_feature,x_train_senti_feature,tf_word_train,posfeature_train,x_train_sim))#tf_word_train,
x_test_f = np.hstack((x_test_glove,x_test_normal_feature,x_test_senti_feature,tf_word_test,posfeature_test,x_test_sim))#,tf_char_test
scaler2 = preprocessing.StandardScaler().fit(x_train_f)
x_train_fn = scaler2.transform(x_train_f)
x_test_fn = scaler2.transform(x_test_f)

In [ ]:
print(x_test_bert.shape)
x_train_re = x_train_bert.reshape((336,3*20000))
x_test_re = x_test_bert.reshape((84,3*20000))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA


In [ ]:
def sv(x_train_fn, y_train,x_test_fn, y_test):
  clf_svm = svm.SVC()
  clf_svm.fit(x_train_fn, y_train)
  y_pred_svmall=clf_svm.predict(x_test_fn)
  print('acc {}'.format(accuracy_score(y_test,y_pred_svmall)))
  print('f1_score {}'.format(f1_score(y_test,y_pred_svmall)))
  return accuracy_score(y_test,y_pred_svmall), f1_score(y_test,y_pred_svmall)

In [ ]:
sv(posfeature_train, y_train,posfeature_test,y_test)
sv(tf_word_train, y_train,tf_word_test,y_test)
sv(np.hstack([x_train_glove,x_train_sim]), y_train,np.hstack([x_test_glove,x_test_sim]),y_test)

In [ ]:
sv(x_train_normal_feature, y_train,x_test_normal_feature,y_test)
sv(x_train_senti_feature, y_train,x_test_senti_feature,y_test)
sv(x_train_fn, y_train,x_test_fn,y_test)

In [ ]:
def LR(x_train_fn, y_train,x_test_fn, y_test):
  lr_all=LogisticRegression(solver="liblinear",random_state=2)
  lr_all.fit(x_train_fn, y_train)
  y_pred_lrall=lr_all.predict(x_test_fn)
  print('acc {}'.format(accuracy_score(y_test,y_pred_lrall)))
  print('f1_score {}'.format(f1_score(y_test,y_pred_lrall)))
  return accuracy_score(y_test,y_pred_lrall), f1_score(y_test,y_pred_lrall)

In [ ]:
LR(posfeature_train, y_train,posfeature_test,y_test)

In [ ]:
LR(tf_word_train, y_train,tf_word_test,y_test)

In [ ]:
LR(np.hstack([x_train_glove,x_train_sim]), y_train,np.hstack([x_test_glove,x_test_sim]),y_test)

In [ ]:
LR(x_train_normal_feature, y_train,x_test_normal_feature,y_test)
LR(x_train_senti_feature, y_train,x_test_senti_feature,y_test)
LR(x_train_fn, y_train,x_test_fn,y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
def RF(x_train_fn, y_train,x_test_fn,y_test):
  clf_all = RandomForestClassifier(max_depth=8, random_state=0)
  clf_all.fit(x_train_fn, y_train)
  y_pred_all=clf_all.predict(x_test_fn)
  print('acc {}'.format(accuracy_score(y_test,y_pred_all)))
  print('f1_score {}'.format(f1_score(y_test,y_pred_all)))
  return accuracy_score(y_test,y_pred_all), f1_score(y_test,y_pred_all)

In [ ]:
RF(posfeature_train, y_train,posfeature_test,y_test)

In [ ]:
RF(tf_word_train, y_train,tf_word_test,y_test)

In [ ]:
RF(np.hstack([x_train_glove,x_train_sim]), y_train,np.hstack([x_test_glove,x_test_sim]),y_test)

In [ ]:
RF(x_train_normal_feature, y_train,x_test_normal_feature,y_test)

In [ ]:
RF(x_train_senti_feature, y_train,x_test_senti_feature,y_test)

In [ ]:
RF(x_train_fn, y_train,x_test_fn,y_test)

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Conv2D,Conv1D
# from keras.layers import Merge
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from theano.tensor.shared_randomstreams import RandomStreams
from keras.models import Model
from keras.layers import Input
from keras.layers import Reshape
from keras import optimizers
from keras.regularizers import l2
from keras.regularizers import l1
# from keras.layers.normalization import BatchNormalization
from keras.constraints import nonneg
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
from sklearn import preprocessing
# scaler = preprocessing.StandardScaler().fit(x_train_feature)
# x_train_feature_n=scaler.transform(x_train_feature)
# x_test_feature_n=scaler.transform(x_test_feature)

scaler_normal = preprocessing.StandardScaler().fit(x_train_normal_feature)
x_train_normal_feature_n=scaler_normal.transform(x_train_normal_feature)
x_test_normal_feature_n=scaler_normal.transform(x_test_normal_feature)

scaler_senti = preprocessing.StandardScaler().fit(x_train_senti_feature)
x_train_senti_feature_n=scaler_senti.transform(x_train_senti_feature)
x_test_senti_feature_n=scaler_senti.transform(x_test_senti_feature)

scaler_pos = preprocessing.StandardScaler().fit(posfeature_train)
x_train_pos_feature_n=scaler_pos.transform(posfeature_train)
x_test_pos_feature_n=scaler_pos.transform(posfeature_test)

scaler_tf = preprocessing.StandardScaler().fit(tf_word_train)
x_train_tf_feature_n=scaler_tf.transform(tf_word_train)
x_test_tf_feature_n=scaler_tf.transform(tf_word_test)

scaler_sim = preprocessing.StandardScaler().fit(x_train_sim)
x_train_sim_n=scaler_sim.transform(x_train_sim)
x_test_sim_n=scaler_sim.transform(x_test_sim)

# x_train_f = np.hstack((x_train_glove,x_train_feature,tf_word_train,posfeature_train))
# x_test_f = np.hstack((x_test_glove,x_test_feature,tf_word_test,posfeature_test))
x_train_f = np.hstack((x_train_glove,x_train_normal_feature,x_train_senti_feature,tf_word_train,posfeature_train,x_train_sim))#tf_word_train,
x_test_f = np.hstack((x_test_glove,x_test_normal_feature,x_test_senti_feature,tf_word_test,posfeature_test,x_test_sim))#,tf_char_test
scaler_full = preprocessing.StandardScaler().fit(x_train_f)
x_train_full_n=scaler_full.transform(x_train_f)
x_test_full_n=scaler_full.transform(x_test_f)

In [ ]:
x0=np.array(x_train_glove)[:,np.newaxis,:]
x1=x_train_normal_feature_n[:,np.newaxis,:]
x2=x_train_senti_feature_n[:,np.newaxis,:]
x3 = x_train_pos_feature_n[:,np.newaxis,:]
x4 = x_train_tf_feature_n[:,np.newaxis,:]
x5=x_train_sim_n[:,np.newaxis,:]
x0_test=x_test_glove[:,np.newaxis,:]
x1_test=x_test_normal_feature_n[:,np.newaxis,:]
x2_test=x_test_senti_feature_n[:,np.newaxis,:]
x3_test = x_test_pos_feature_n[:,np.newaxis,:]
x4_test = x_test_tf_feature_n[:,np.newaxis,:]
x5_test=x_test_sim_n[:,np.newaxis,:]
y_train_1=y_train[:,np.newaxis]
y_test_1=y_test[:,np.newaxis]

In [ ]:
def build_model_fusion(exits, num_aux_targets):
    # ipt1=input[0]
    # ipt2=input[1]
    # ipt3=input[2]
    # ipt4=input[3]
    # ipt5=input[4]

    ipt0 = Input((1,50))
    # bp0 = LSTM(128)(ipt0)
    # bp0 = LSTM(128)(ipt0)
    # bp0=Dropout(0.4)(bp0)
    # x=layers.Conv1D(128,64)()
    # x=layers.MaxPooling1D()(x)
    # x=Dropout(0.5)(x)                 
    # x=GlobalAveragePooling1D()(x)
    bp0 = Dense(64, activation='relu')(ipt0)
    bp0=Dropout(0.5)(bp0)
    d0 = Dense(64, activation='relu')(bp0)

    ipt1 = Input((1,6))
    # x = LSTM(10)(ipt1)
    # x=Dropout(0.3)(x)

    x = Dense(32, activation='relu')(ipt1)
    x= Dropout(0.5)(x)
    d1 = Dense(8, activation='relu')(x)

    ipt2=Input((1,16))
    # bp2=LSTM(32)(ipt2)
    # bp2=Dropout(0.3)(bp2)
    bp2=Dense(32,activation="relu")(ipt2)
    bp2=Dropout(0.5)(bp2)
    d2=Dense(16,activation="relu")(bp2)

    ipt3=Input((1,30))
    # bp3=LSTM(32)(ipt3)
    # bp3=Dropout(0.3)(bp3)
    bp3=Dense(64,activation="relu")(ipt3)
    bp3=Dropout(0.5)(bp3)
    d3=Dense(32,activation="relu")(bp3)

    ipt4=Input((1,32))
    # bp4=LSTM(64)(ipt4)
    # bp4=Dropout(0.3)(bp4)
    bp4=Dense(32,activation="relu")(ipt4)
    bp4=Dropout(0.5)(bp4)
    d4=Dense(32,activation="relu")(bp4)

    ipt5=Input((1,2))
    # bp5=LSTM(32)(ipt5)
    # bp5=Dropout(0.3)(bp5)
    bp5=Dense(32,activation="relu")(ipt5)
    bp5=Dropout(0.5)(bp5)
    d5=Dense(16,activation="relu")(bp5)

    a=4
    b=8
    if exits==[0]:
      m=tf.keras.layers.Concatenate(axis=-1)([d0,d5])
      # m=d
    if exits==[1]:
      m=d1
    if exits==[2]:
      m=d2
    if exits==[3]:
      m=d3
    if exits==[4]:
      m=d4
    if exits==[0,1,2,3,4]:
      m=tf.keras.layers.Concatenate(axis=-1)([d0,d1,d2,d3,d4,d5])
    # print(m.shape)
    merged=Dropout(0.4)(m)
    merged=Dense(128,activation='relu', kernel_regularizer=l2(0.02), name = 'fusion_layer_1')(merged)
    merged=Dropout(0.4)(merged)
    merged=Dense(128, activation='relu', kernel_regularizer=l2(0.01), name = 'fusion_layer_2')(merged)
    # merged += m
    merged=Dropout(0.5)(merged)
    result=Dense(1, activation='sigmoid',kernel_regularizer=l2(0.005))(merged)
    # result=Dense(1, weights=[np.array([[b]]),np.array([-a])], trainable=False)
    # aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    
    train_patience=5
    train_epoch=10
    weights_path="/content/drive/MyDrive/nlp"
    callbacks = [
      EarlyStopping(monitor='val_loss', patience=train_patience, verbose=0),
      ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True, verbose=0),
    ]
    adm = keras.optimizers.Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    sgd = SGD(learning_rate=0.04, decay=1e-6, momentum=0.1, nesterov=True)

    fusion_model = Model(inputs=[ipt0,ipt1,ipt2,ipt3,ipt4,ipt5], outputs=result)
    fusion_model.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['accuracy'])
    fusion_model.run_eagerly = True

    return fusion_model

In [ ]:
import matplotlib.pyplot as plt
checkpoint_predictions_1 = []
weights_1 = []
y_num=2
NUM_MODELS=1
EPOCHS=25
best_acc=0
loss_train1=[]
acc_train1=[]
loss_test1=[]
acc_test1=[]
for model_idx in range(NUM_MODELS):
    model = build_model_fusion([0,1,2,3,4],y_train.shape[0])
    gg=0
    for global_epoch in range(EPOCHS):
        if (global_epoch%20) ==0:
          gg+=1
        history=model.fit(
            [x0,x1,x2,x3,x4,x5],
            y_train_1,
            validation_data=([x0_test,x1_test,x2_test,x3_test,x4_test,x5_test],y_test_1),
            batch_size=8,
            epochs=1,
            verbose=2

            # callbacks=[
            #     EarlyStopping(monitor='val_loss', patience=5, verbose=0),
            #     LearningRateScheduler(lambda epoch: 0.05-(0.005 * (gg-1)))
            # ]
        )
        if best_acc < history.history['val_accuracy'][0]:
          best_acc=history.history['val_accuracy'][0]
          model.save('/content/drive/MyDrive/nlp/my_h5_model.h5')

        loss_train1.append(history.history['loss'])
        acc_train1.append(history.history['accuracy'])
        loss_test1.append(history.history['val_loss'])
        acc_test1.append(history.history['val_accuracy'])
        checkpoint_predictions_1.append(model.predict([x0_test,x1_test,x2_test,x3_test,x4_test,x5_test], batch_size=8))
        weights_1.append(2 ** global_epoch)

predictions = np.average(checkpoint_predictions_1, weights=weights_1, axis=0)
model=keras.models.load_model('/content/drive/MyDrive/nlp/my_h5_model.h5')
pp=model.predict([x0_test,x1_test,x2_test,x3_test,x4_test,x5_test])
ppp=pp.copy()
ppp[pp<0.5]=0
ppp[pp>=0.5]=1
ppp=np.squeeze(ppp)
print("acc {}".format(accuracy_score(ppp, y_test)))
print("f1 {}".format(f1_score(ppp, y_test)))
# model.fit([x0,x1,x2,x3,x4,x5],y_train,validation_data=([x0_test,x1_test,x2_test,x3_test,x4_test,x5_test],y_test),epochs=20,batch_size=8)

plt.figure(figsize=(18,6))
plt.subplot(121)
plt.plot(loss_train1)
plt.plot(loss_test1)
plt.title('Feature-level Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

In [ ]:
# model.fit(xxx,y_train,validation_data=(xxxx,y_test),epochs=10,batch_size=8)
def model2(xx_test, xx_train, y_test, y_train,path):
  model = tf.keras.Sequential([ #build the Bi-LSTM model
      # tf.keras.layers.Embedding(1,136),
      # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True)),
      # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
      tf.keras.layers.Dense(256,activation='relu',kernel_regularizer=l2(0.01)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(128,activation='relu',kernel_regularizer=l2(0.01)),
      tf.keras.layers.Dropout(0.5),
      # tf.keras.layers.Dense(128,activation='relu',kernel_regularizer=l2(0.01)),
      # tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(64,activation='relu',kernel_regularizer=l2(0.007)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  adm = keras.optimizers.Adam(learning_rate=0.01, epsilon=1e-07)#use the momentum sgd
  sgd = SGD(learning_rate=0.02, decay=1e-6, momentum=0.1, nesterov=True)
  model.compile(loss='binary_crossentropy',optimizer=adm, metrics=['accuracy'])#compile the model
  model.run_eagerly = True


  checkpoint_predictions = []
  weights = []
  y_num=2
  NUM_MODELS = 1
  BATCH_SIZE = 512
  LSTM_UNITS = 128
  DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
  EPOCHS = 10
  MAX_LEN = 22
  best_acc=0
  loss_train=[]
  acc_train=[]
  loss_test=[]
  acc_test=[]
  for model_idx in range(NUM_MODELS):
      for global_epoch in range(EPOCHS):
        history=model.fit(
              xx_train,
              y_train,
              batch_size=8,
              epochs=1,
              verbose=2,
              validation_data=(xx_test,y_test),
              callbacks=[
                  LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))
              ]
          )
        if best_acc < history.history['val_accuracy'][0]:
          best_acc=history.history['val_accuracy'][0]
          model.save('/content/drive/MyDrive/nlp/my_h5_model2.h5')

        loss_train.append(history.history['loss'])
        acc_train.append(history.history['accuracy'])
        loss_test.append(history.history['val_loss'])
        acc_test.append(history.history['val_accuracy'])

        checkpoint_predictions.append(model.predict(xx_test, batch_size=8))
        weights.append(2 ** global_epoch)

  model=keras.models.load_model('/content/drive/MyDrive/nlp/my_h5_model2.h5')
  predictions = np.average(checkpoint_predictions, weights=weights, axis=0)
  pp=model.predict(xx_test)
  ppp=pp.copy()
  ppp[pp<0.5]=0
  ppp[pp>=0.5]=1
  ppp=np.squeeze(ppp)
  print("acc {}".format(accuracy_score(ppp, y_test)))
  print("f1 {}".format(f1_score(ppp, y_test)))
  # model.fit(x_test_fn,y_train,validation_data=(x_test_fn,y_test),epochs=10,batch_size=8)
  plt.figure(figsize=(18,6))
  plt.subplot(121)
  plt.plot(loss_train)
  plt.plot(loss_test)
  plt.title('Model-level Loss',fontsize=15)
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [ ]:
model2(x_test_fn, x_train_fn, y_test, y_train,'/content/drive/MyDrive/nlp/model2.jpg')

In [ ]:
COLS=[]
nomal=['word_length', 'woken_length', 'avg_token_length', 'capitalized', 'specific', 'intensifier']
sen=['Emoji_positive', 'Emoji_negative', 'Emoji_neutral', 'subjective_weak', 'subjective_strong', 'subjective_positive', 'subjective_negative', 'subjlexicon_neutral', 'subjective_total', 'swn_positive', 'swn_negative', 'swn_compound', 'vad_positive', 'vad_negative', 'vad_neutral', 'vad_compound']

for i in range(50):
  COLS.append('glove'+str(i+1))
COLS=COLS+nomal+sen
for i in range(72,102):
  COLS.append('pos'+str(i-71))
for i in range(102,134):    
  COLS.append('tfidf'+str(i-101))
COLS.append('max_similar')
COLS.append('min_similar')

In [ ]:
x_fffff=x_train_full_n[:,np.newaxis,:]
x_test_fffff=x_test_full_n[:,np.newaxis,:]

In [ ]:
TYPE=['Semantic']*50
TYPE=TYPE+['Normal']*6+['Sentiment']*16+['Syntactic']*30+['Lexical']*32+['Semantic']*2

In [ ]:

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
 
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from IPython.display import display
 
COMPUTE_LSTM_IMPORTANCE = 1
ONE_FOLD_ONLY = 1
NUM_FOLDS=5
 
# with gpu_strategy.scope():
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
test_preds = []
results_all=[]
for fold, (train_idx, test_idx) in enumerate(kf.split(x_train_full_n, y_train)):
    K.clear_session()
    
    print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
    X_train, X_valid = x_fffff[train_idx], x_fffff[test_idx]
    Y_train1, y_valid = y_train[train_idx], y_train[test_idx]

# for i in range(50):
#   COLS.append('glove'+str(i+1))
# COLS=COLS+nomal+sen
# for i in range(72,102):
#   COLS.append('pos'+str(i-71))
# for i in range(102,134):    
#   COLS.append('tfidf'+str(i-101))
# COLS.append('max_similar')
# COLS.append('min_similar')   
    # 导入已经训练好的模型
    model = keras.models.load_model('/content/drive/MyDrive/nlp/my_h5_model-1.h5')
    # 计算特征重要性
    if COMPUTE_LSTM_IMPORTANCE:
        results = []
        print(' Computing LSTM feature importance...')

        for k in tqdm(range(len(COLS))):
            # if k>0: 
            save_col = X_valid[:,:,k].copy()
            np.random.shuffle(X_valid[:,:,k])
            # save_col = X_valid[:,:,k-1].copy()
            # np.random.shuffle(X_valid[:,:,k-1])
            x0_f,x1_f,x2_f,x3_f,x4_f,x5_f=X_valid[:,:,:50],X_valid[:,:,50:56],X_valid[:,:,56:72],X_valid[:,:,72:102],X_valid[:,:,102:134],X_valid[:,:,134:]
            oof_preds = model.predict([x0_f,x1_f,x2_f,x3_f,x4_f,x5_f], verbose=0).squeeze() 
            loss=np.mean(y_valid*np.log(oof_preds)+(1-y_valid)*np.log(1-oof_preds))
            # mse = np.mean(( oof_preds-y_valid )**2)
            # results.append({'feature':COLS[k],'loss':loss})  
            results.append(loss)
            if k>0: 
                X_valid[:,:,k-1] = save_col
        results_all.append(results)
results=np.mean(results_all,axis=0)


In [ ]:
df={'features':COLS,'loss':-results}
df=pd.DataFrame(df)
df = df.sort_values('loss')
plt.figure(figsize=(10,20))
plt.barh(np.arange(len(COLS)),df.loss)
plt.yticks(np.arange(len(COLS)),df.features.values)
plt.title('BiLSTM Feature Importance',size=16)
plt.ylim((-1,len(COLS)))
plt.show()
                    
# SAVE LSTM FEATURE IMPORTANCE
# df = df.sort_values('loss',ascending=False)
# df.to_csv(f'bilstm_feature_importance_fold_{fold}.csv',index=False)

In [ ]:
TYPE=['Semantic']*50
TYPE=TYPE+['Normal']*6+['Sentiment']*16+['Syntactic']*30+['Lexical']*32+['Semantic']*2

df={'type':TYPE,'features':COLS,'loss':results}
colors={'Normal':'#CD5C5C','Sentiment':'#00BFFF','Syntactic':'orange','Semantic':'#696969','Lexical':'#32CD32'}
df=pd.DataFrame(df)
# df = df.sort_values('loss')
fea_more=nomal+sen+['tfidf','glove','pos']+['max_similar','min_similar']
cate=['Normal']*6+['Sentiment']*16+['Lexical','Semantic','Syntactic']+['Semantic']*2
loss_avg=[]
for x in fea_more:
  bool_index = df.features.str.contains(x)
  l=np.mean(df[bool_index]['loss'])
  loss_avg.append(l)
s=np.flipud(np.argsort(loss_avg))
fea_more=np.array(fea_more)[s]
cate=np.array(cate)[s]
loss_avg=np.array(loss_avg)
loss_avg=abs(np.sort(-loss_avg))
plt.figure(figsize=(12,10))

# abels = df['Condition'].unique()


plt.barh(np.arange(len(fea_more)),loss_avg,height=0.5, color=[colors[i] for i in cate])
plt.yticks(np.arange(len(fea_more)),fea_more)
plt.title('Permutation Feature Importance',size=15)
plt.ylabel('features',size=10)
plt.xlabel('absolute binary crossentropy loss',size=15)
plt.ylim((-1,len(fea_more)))
plt.xlim((min(loss_avg)-0.4,1.15))
cate_u=list(set(cate))
handles = [plt.Rectangle((0,0),1,1, color=colors[l]) for l in cate_u]
plt.legend(handles, cate_u)
plt.savefig('importance.jpg',dpi=600)
plt.show()
# bool_index1 = df.features.str.contains('tfidf')
# bool_index2 = df.features.str.contains('senti')
# bool_index3 = df.features.str.contains('glove')
# bool_index4 = df.features.str.contains('pos')
# bool_index5 = df.features.str.contains('normal')
# loss_avg.append(bool_index)